In [169]:
import csv
import math
import requests
import urllib2
from bs4 import BeautifulSoup as BS
import numpy as np


In [145]:
#Please download key ratios csv from morningstar.in and give its name below
def hitMorningStar(Type, companyCode):
    if(Type=='keyRatio'):
        morningStarURL = "http://financials.morningstar.com/ajax/exportKR2CSV.html?t="
        morningStarURL += companyCode
    else:
        morningStarURL = "http://financials.morningstar.com/ajax/ReportProcess4CSV.html?t="+companyCode+"&reportType=bs&period=12&dataType=A&order=asc&columnYear=5&number=3"
    #hit API here and download csv.

    with requests.Session() as s:
        download = s.get(morningStarURL)
        decoded_content = download.content
        cr = csv.reader(decoded_content.splitlines(), delimiter=',')
        mylist = list(cr)
        
    return mylist

In [146]:
def convToFloat(var):
    if(var=='' or var=='-'):
        var=float(0.0)
    else:
        n=len(var)
        rstr=""
        for ctr in range(0,n):
            #print(ctr)
            if(var[ctr]!=','):
                rstr=rstr+var[ctr]
        var=float(rstr)
    return(var)


In [147]:
def makeRequest(url):
    req = urllib2.Request(url,headers={'User-Agent': 'Mozilla/5.0'})
    html = urllib2.urlopen(req).read()
    soup = BS(html, "html.parser")
    return soup

In [148]:
def getCurrentDetails(companyName):
    url='https://in.investing.com/equities/'+companyName
    #req = urllib.request.Request('https://in.investing.com/equities/havells-india', headers={'User-Agent': 'Mozilla/5.0'}) #VERY IMP

    soup = makeRequest(url)
    currentDetails = {}
    
    #to get last trade price
    ltp=soup.find(class_="newInput inputTextBox alertValue")
    ltp=convToFloat(ltp['value'])
    currentDetails['ltp'] = ltp
    print('ltp - ',ltp)
    
    #to get name
    scrip=soup.find(class_="float_lang_base_1 relativeAttr").get_text()
    currentDetails['scrip'] = scrip
    print (scrip)

    #to get industry
    industry=soup.find(class_="companyProfileHeader").get_text()
    currentDetails['industry'] = industry
    print (industry)

    #to get market cap
    mcap=soup.find_all(class_="float_lang_base_2 bold")
    marketcap=(mcap[7]).get_text()
    currentDetails['marketCap'] = marketcap
    print (marketcap)

    return currentDetails, ltp, scrip, industry, marketcap
    

In [156]:
def parseBalanceSheet(companyCode):
    #balanceSheetURL='https://in.investing.com/equities/'+companyName+'-balance-sheet'
    mylist = hitMorningStar('bs',companyCode)
    #print balanceSheetURL
    #Enter balance sheet
    #soup = makeRequest(balanceSheetURL)
    #balanceSheet= soup.find_all(id="parentTr")

    #to parse Total Current assets
    req=balanceSheet[0]
    ca=[]
    for record in req.findAll('td'):
        ca.append(str(record.text))
    currentAssets=ca[2]
    currentAssets=convToFloat(currentAssets)/10 #WHY?
    print(currentAssets)


    #to parse Total Total Assets
    req=balanceSheet[1]
    ta=[]
    for record in req.findAll('td'):
        ta.append(str(record.text))
    totalAssets=ta[2]
    totalAssets=convToFloat(totalAssets)/10
    print(totalAssets)

    #to parse Current Liabilities
    req=balanceSheet[2]
    cl=[]
    for record in req.findAll('td'):
        cl.append(str(record.text))
    currentLiabilities=cl[2]
    currentLiabilities=convToFloat(currentLiabilities)/10
    print(currentLiabilities)

    #to parse Total Liabilities
    req=balanceSheet[3]
    tl=[]
    for record in req.findAll('td'):
        tl.append(str(record.text))
    totalLiabilities=tl[2]
    if totalLiabilities== "-":
        totalLiabilities="0"
    totalLiabilities=float(totalLiabilities)/10
    print(totalLiabilities)

    #to parse Total Equity
    req=balanceSheet[4]
    te=[]
    for record in req.findAll('td'):
        te.append(str(record.text))
    totalEquity=te[2]
    if totalEquity== "-":
        totalEquity="0"
    totalequity=float(totalEquity)/10
    print(totalEquity)

    #to parse goodwill
    balanceSheet= soup.find_all(class_="child")
    req=balanceSheet[12]
    gw=[]
    for record in req.findAll('td'):
        gw.append(str(record.text))
    goodWill=gw[2]
    if goodWill== "-":
        goodWill="0"
    goodWill=float(goodWill)/10
    print(goodWill)

    #to parse intangibles
    req=balanceSheet[13]
    inta=[]
    for record in req.findAll('td'):
        inta.append(str(record.text))
    intangibles=inta[2]
    if intangibles== "-":
        intangibles="0"
    intangibles=float(intangibles)/10
    print(intangibles)

    return currentAssets, totalAssets, totalLiabilities, totalEquity, goodWill, intangibles

In [160]:
def writeCSVRow(companyName, companyCode):
    mylist = hitMorningStar('keyRatio',companyCode)
    currentDetails, ltp, scrip, industry, marketcap = getCurrentDetails(companyName)
    currentAssets, totalAssets, totalLiabilities, totalEquity, goodWill, intangibles = parseBalanceSheet(companyName)
    

    #print(balsh[12])
    #value1= soup.find_all('span')
    #print(value1)
    #a=value1[0].index("Total Common Shares Outstanding")
    #print(mylist)

    #a is dummy variable
    a=mylist[2][0]

    nshares=(convToFloat(mylist[11][10]))/10
    #print(nshares)

    eps=float(mylist[8][10])
    #print(eps)

    pe=ltp/eps
    cacl=currentAssets/currentLiabilities
    ca=1.1*currentAssets
    nta=totalAssets-goodWill-intangibles
    ntas=nta/nshares
    if(ntas!=0):
        pnta=ltp/ntas
    else:
        pnta=0


    ltde=convToFloat(mylist[99][10])

    roe1=convToFloat(mylist[38][6])
    roe2=convToFloat(mylist[38][7])
    roe3=convToFloat(mylist[38][8])
    roe4=convToFloat(mylist[38][9])
    roe5=convToFloat(mylist[38][10])
    roe5ya=(roe1+roe2+roe3+roe4+roe5)/5
    ni5y=convToFloat(mylist[56][10])
    rg10y=convToFloat(mylist[47][10])
    eps10y=convToFloat(mylist[62][10])
    #print(mylist[9][0])
    #print(mylist[12][0])
    d8=convToFloat(mylist[9][1])
    d9=convToFloat(mylist[9][2])
    d10=convToFloat(mylist[9][3])
    d11=convToFloat(mylist[9][4])
    d12=convToFloat(mylist[9][5])
    d13=convToFloat(mylist[9][6])
    d14=convToFloat(mylist[9][7])
    d15=convToFloat(mylist[9][8])
    d16=convToFloat(mylist[9][9])
    d17=convToFloat(mylist[9][10])
    b8=convToFloat(mylist[12][1])
    b9=convToFloat(mylist[12][2])
    b10=convToFloat(mylist[12][3])
    b11=convToFloat(mylist[12][4])
    b12=convToFloat(mylist[12][5])
    b13=convToFloat(mylist[12][6])
    b14=convToFloat(mylist[12][7])
    b15=convToFloat(mylist[12][8])
    b16=convToFloat(mylist[12][9])
    b17=convToFloat(mylist[12][10])
    if(b17<1 or b8<1):
        bvchange=10
    else:
        bvchange=((math.pow((b17/b8),0.1))-1)*100

    defInterestRate = 6.25
    FVofDividend = np.fv(defInterestRate/100.0,10,-1*d17,-1*d17,0)
    FVofBV = np.fv(bvchange/100.0,10,0,-1*b17)
    Total = FVofDividend + FVofBV
    IntrinsicValue = np.pv(defInterestRate/100.0,10,0,Total,0)
    PercentageIncrease = 100*(IntrinsicValue - ltp)/ltp
    #print(nshares)
    nlist=[[scrip,industry,nshares,marketcap,ltp,eps,pe,totalAssets,currentAssets,currentLiabilities,cacl,totalLiabilities,ca,goodWill,intangibles,nta,ntas,pnta,ltde,roe1,roe2,roe3,roe4,roe5,roe5ya,ni5y,rg10y,eps10y,d8,d9,d10,d11,d12,d13,d14,d15,d16,d17,b8,b9,b10,b11,b12,b13,b14,b15,b16,b17,bvchange,    FVofDividend, FVofBV, Total, IntrinsicValue, PercentageIncrease]]
    #print(nlist)
    #all data will be put into this csv file. This file is overwritten every time you run the code.
    my_new_list = open('stock.csv', 'a')
    csv_writer = csv.writer(my_new_list)
    csv_writer.writerows(nlist)
    my_new_list.close()

In [161]:
companyCode = "ALLCARGO"
companyName = "allcargo-logistics"

In [162]:
writeCSVRow(companyName, companyCode)

('ltp - ', 189.8)
Allcargo Logistics Ltd (ACLL)	

IndustryBusiness Services
SectorServices
Employees4423
Equity TypeORD

46.63B
https://in.investing.com/equities/allcargo-logistics-balance-sheet
0.0
0.0
0.0
0.0
0
0.0
0.0


In [167]:
parseBalanceSheet("reliance-industries")

https://in.investing.com/equities/reliance-industries-balance-sheet
161804.0
752024.0
277636.0
475715.0
2763090
4639.0
103305.0


(161804.0, 752024.0, 475715.0, '2763090', 4639.0, 103305.0)